In [0]:
# Este código se ejecuta en una celda de un Notebook de Databricks
# Su responsabilidad es enriquecer la tabla de ventas con la dimensión de productos
# para crear una tabla de hechos (fact table) en la Capa de Oro (Gold).

from pyspark.sql.functions import col

# 1. --- Configuración de Tablas ---
# Tablas de entrada
sales_silver_table = "workspace.tecnomundo_data_processed.reporte_de_ventas_por_articulos_2_cleaned"
products_dim_table = "workspace.tecnomundo_data_gold.dim_products"

# Tabla de salida en la capa de Oro
output_gold_table = "workspace.tecnomundo_data_gold.fact_sales"

print(f"Tabla de Ventas (Plata): {sales_silver_table}")
print(f"Tabla de Dimensión de Productos (Oro): {products_dim_table}")
print(f"Tabla de Destino (Oro): {output_gold_table}")


# 2. --- Lectura de las Tablas de Entrada ---
print("\nLeyendo tablas de entrada...")
try:
    df_sales = spark.table(sales_silver_table)
    df_products = spark.table(products_dim_table)
    
    print(f"Se leyeron {df_sales.count()} registros de la tabla de ventas.")
    print(f"Se leyeron {df_products.count()} registros de la tabla de dimensión de productos.")

except Exception as e:
    print(f"Error al leer las tablas de entrada: {e}")
    dbutils.notebook.exit("No se pudieron leer las tablas de origen.")


# 3. --- Enriquecimiento de la Tabla de Ventas ---
print("\nEnriqueciendo las ventas con los datos de la dimensión de productos...")

# Hacemos un LEFT JOIN desde las ventas hacia la dimensión de productos.
df_fact_sales = df_sales.join(
    df_products,
    df_sales["codigo_producto"] == df_products["codigo_producto"],
    "left"
).select(
    # Seleccionamos todas las columnas originales de la tabla de ventas
    df_sales["*"],
    # Y añadimos las columnas de enriquecimiento desde la dimensión de productos
    df_products.nombre_del_producto,
    df_products.categoria,
    df_products.stock_actual
)

print("Join completado.")


# 4. --- Guardado en la Capa de Oro ---
print(f"\nGuardando la tabla de hechos de ventas en '{output_gold_table}'...")

(df_fact_sales.write
 .mode("overwrite")
 .option("overwriteSchema", "true")
 .saveAsTable(output_gold_table))

print("¡Tabla de hechos de ventas guardada exitosamente en la Capa de Oro!")


# 5. --- Verificación Final ---
print("\nMostrando una muestra de la nueva tabla de hechos enriquecida:")
display(spark.table(output_gold_table))

print("\nEsquema final de la tabla de hechos:")
spark.table(output_gold_table).printSchema()
